In [ ]:
import os
import re
import sys
import glob
import pickle
import shelve
from collections import OrderedDict
from sklearn.metrics import mean_absolute_percentage_error as mape

from comet_ml.api import API, APIExperiment

import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib
from matplotlib import rcParams
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf
from tensorflow import keras

if '..' not in sys.path:
    sys.path.append('..')
from dlml.utils import collect_experiments
from dlml.data import read_area_values, load_data_areas
from dlml.nn import predict

In [ ]:
# full grid without varying compensator's inertia
experiment_ID = 'f64bde90cab54d1ea770bb21f33c3ed1'
# full grid with variable compensator's inertia
experiment_ID = 'a40658acee3c4e419c0ee34d0c59f4df'

workspace = 'danielelinaro'
project_name = 'inertia'

api = API(api_key = os.environ['COMET_API_KEY'])
experiment = api.get_experiment(workspace, project_name, experiment_ID)
tags = experiment.get_tags()

print(f'\nExperiment {experiment_ID[:6]} has the following tags:')
nchar = 0
for tag in tags:
    if 'buses_' in tag:
        rec_bus_IDs = sorted(list(map(int, re.findall('\d+', tag))))
    elif 'area_measure_' in tag:
        area_measure = tag.split('_')[-1]
    elif 'area' in tag:
        area_ID = int(tag[4:])
    elif tag == 'all_stoch_loads':
        stoch_load_bus_IDs = []
    sys.stdout.write('"' + tag + '"   ')
    nchar += len(tag) + 5
    if nchar >= 70:
        nchar = 0
        sys.stdout.write('\n')
print('')

In [ ]:
experiment_dir = os.path.join('..', 'experiments', 'neural_network', experiment_ID)
weights_file = os.path.join(experiment_dir, 'checkpoints', 'weights.h5')
model = keras.models.load_model(weights_file, compile=False)
model.compile()
network_pars = pickle.load(open(os.path.join(experiment_dir, 'parameters.pkl'), 'rb'))
data_dirs = [os.path.join('..', d.format(a)) if '{}' in d else os.path.join('..', d) \
             for d,a in zip(network_pars['data_dirs'], network_pars['area_IDs'])]
# we need mean and standard deviation of the training set to normalize the data
x_train_mean = network_pars['x_train_mean']
x_train_std  = network_pars['x_train_std']
data_dir = data_dirs[0]
if not os.path.isdir(data_dir):
    raise Exception(f'{data_dir}: no such directory')

print(f'Loaded network from {weights_file}.')
print(f'Data directory is {data_dir}.')

In [ ]:
set_name = 'test'
data_files = {set_name: glob.glob(os.path.join(*data_dir.split(os.path.sep)[:-1],
                                               'wide_grid_test', f'inertia_*_{set_name}_set.h5'))}
_,X_raw,y = load_data_areas(data_files,
                            network_pars['var_names'],
                            network_pars['generators_areas_map'],
                            network_pars['generators_Pnom'],
                            network_pars['area_measure'],  
                            trial_dur=network_pars['trial_duration'],
                            max_block_size=network_pars['max_block_size'],
                            use_fft=network_pars['use_fft'],
                            use_tf=False,
                            verbose=False)
X_test = tf.constant([(X_raw[set_name][i] - m) / s for i,(m,s) in enumerate(zip(x_train_mean, x_train_std))])
y_test = y[set_name][:,0]
y_predict = model.predict({var_name: X_test[i] for i,var_name in enumerate(network_pars['var_names'])}).squeeze()

In [ ]:
n_files = len(data_files[set_name])
block_size = y_test.shape[0] // n_files
data = {'H_G02': [], 'H_G03': [], 'M_area1': [], 'M_area1_pred_mean': [], 'M_area1_pred_std': [], 'MAPE': []}
for i,data_file in enumerate(data_files[set_name]):
    names, H, M, _ = read_area_values(data_file,
                                      network_pars['generators_areas_map'],
                                      network_pars['generators_Pnom'],
                                      network_pars['area_measure'])
    for j,name in enumerate(('G02','G03')):
        key = 'H_' + name
        data[key].append(H[j+1])
    data['M_area1'].append(M[0])
    idx = slice(i*block_size, (i+1)*block_size)
    data['M_area1_pred_mean'].append(y_predict[idx].mean())
    data['M_area1_pred_std'].append(y_predict[idx].std())
    data['MAPE'].append(mape(y_test[idx], y_predict[idx]))
df = pd.DataFrame(data)
df.to_parquet(f'wide_grid_test_{experiment_ID[:6]}.parquet.gz', compression='gzip')

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(5,4))
x = [3.33, 5.33, 5.33, 3.33, 3.33]
y = [3.47, 3.47, 5.47, 5.47, 3.47]
ax.plot(x, y, '-', color=.7+np.zeros(3), lw=2.5)
ax.fill_between(x[:2], y[:2], y[2:4], fc=.8+np.zeros(3), alpha=0.5)
im = ax.scatter(df['H_G02'], df['H_G03'], s=df['MAPE']*1000, c=df['MAPE'],
           cmap='RdYlBu_r', vmin=0.0, vmax=0.6)
cbar = plt.colorbar(im)
cbar.set_label('MAPE')
ticks = np.r_[0 : 0.65 : 0.1]
cbar.set_ticks(ticks)
cbar.set_ticklabels([f'{tick*100:.0f}' for tick in ticks])
lim = [1.2,10.8]
ax.set_xlim(lim)
ax.set_ylim(lim)
ticks = np.arange(2, 11, 2)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xlabel(r'$H_{G2}$ [s]')
ax.set_ylabel(r'$H_{G3}$ [s]')
sns.despine()